---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Define a function to get the degree distribution
    def degree_distribution(graph):
        degrees = {}
        
        for n in graph.nodes():
            degree = graph.degree(n)
            
            if degree not in degrees:
                degrees[degree] = 0
                
            degrees[degree] += 1
            
        return sorted(degrees.items())
    
    # Make an empty list to enter the graph identifications
    methods = []
    
    # For every graph, find the degree distribution and average clustering, to identify the algorithm
    for graph in P1_Graphs:
        
        degree_dist = degree_distribution(graph)
        clustering = nx.average_clustering(graph)
        
        if len(degree_dist) > 10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')

    return methods

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.linear_model import LogisticRegression

def salary_predictions():
    
    # For each node, get the 'Department' and 'ManagementSalary' attributes, and put them in a list
    nodes = [(n[0], n[1]['Department'], n[1]['ManagementSalary']) for n in G.nodes(data=True)]
    
    # Convert the nodes with attributes to a pandas DataFrame
    email_df = pd.DataFrame(data=nodes, columns=['Node', 'Department', 'ManagementSalary'])
    email_df.drop('Node', axis=1, inplace=True)
    
    # Add some calculated columns to the DataFrame
    email_df['Degree'] = pd.Series(nx.degree(G))
    email_df['Clustering'] = pd.Series(nx.clustering(G))
    email_df['Degree Centrality'] = pd.Series(nx.degree_centrality(G))
    email_df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    email_df['Betweenness'] = pd.Series(nx.betweenness_centrality(G))
    email_df['Page Rank'] = pd.Series(nx.pagerank(G))
    
    # Split the DataFrame into training data (with known ManagementSalary) and validation data (unknown ManagementSalary)
    train_df = email_df[~pd.isnull(email_df['ManagementSalary'])]
    validate_df = email_df[pd.isnull(email_df['ManagementSalary'])]
    
    # Split the independent variables (X) from the dependent variable (y)
    X = ['Department', 'Degree', 'Clustering', 'Degree Centrality', 'Closeness', 'Betweenness', 'Page Rank']
    X_train = train_df[X]
    y_train = train_df['ManagementSalary']
    
    X_validate = validate_df[X]
    
    # Fit a Logistic Regression model using the training data
    reg = LogisticRegression().fit(X_train, y_train)
    
    # Get the predictions for the unknown individuals
    predictions = reg.predict_proba(X_validate)[:, 1]
    
    return pd.Series(data=predictions, index=X_validate.index)

salary_predictions()

1       0.152228
2       0.589520
5       0.963403
8       0.131627
14      0.355109
18      0.182110
27      0.275287
30      0.329638
31      0.192164
34      0.137055
37      0.126187
40      0.250036
45      0.179844
54      0.254431
55      0.199935
60      0.217215
62      0.995310
65      0.446181
77      0.073155
79      0.125655
97      0.102823
101     0.082812
103     0.194135
108     0.126731
113     0.473462
122     0.066718
141     0.430145
142     0.914255
144     0.092104
145     0.422949
          ...   
913     0.069138
914     0.091397
915     0.016359
918     0.116427
923     0.056263
926     0.089931
931     0.073911
934     0.040544
939     0.028118
944     0.018723
945     0.053540
947     0.104807
950     0.151915
951     0.062855
953     0.048502
959     0.014794
962     0.016574
963     0.152627
968     0.092237
969     0.085329
974     0.067886
984     0.039068
987     0.098152
989     0.085306
991     0.110324
992     0.015900
994     0.018026
996     0.0168

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    
    # Make a new DataFrame, using a tuple of all possible node connections as the index
    df = pd.DataFrame(index=[(n[0], n[1]) for n in nx.preferential_attachment(G)])
    
    # Add some calculated columns on distance measures to use as features
    df['Preferential Attachment'] = [n[2] for n in nx.preferential_attachment(G)]
    df['Soundarjan Hopcroft Count'] = [n[2] for n in nx.cn_soundarajan_hopcroft(G, community='Department')]
    df['Soundarjan Hopcroft RAI'] = [n[2] for n in nx.ra_index_soundarajan_hopcroft(G, community='Department')]
    df['Resource Allocation Index'] = [n[2] for n in nx.resource_allocation_index(G)]
    df['Jaccard'] = [n[2] for n in nx.jaccard_coefficient(G)]
    df['Adamic Adar'] = [n[2] for n in nx.adamic_adar_index(G)]
    df['Common Neighbors'] = [len(list(nx.common_neighbors(G, n[0], n[1]))) for n in df.index.values]
    
    # Join the DataFrame with the 'future_connections' DataFrame using both indexes
    df = df.join(future_connections, how='left')
    
    # Split the DataFrame into training data (known 'Future Connection') and validation data (unknown 'Future Connection')
    train_df = df[~pd.isnull(df['Future Connection'])]
    validate_df = df[pd.isnull(df['Future Connection'])]
    
    # Split the independent variables (X) from the dependent variable (y='Future Connection')
    X_train = train_df.drop('Future Connection', axis=1)
    y_train = train_df['Future Connection']
    
    X_validate = validate_df.drop('Future Connection', axis=1)
    
    # Fit a Logistic Regression model on the training data
    reg = LogisticRegression().fit(X_train, y_train)
    
    # Get the predictions for the unknown individuals
    predictions = reg.predict_proba(X_validate)[:, 1]
    
    return pd.Series(data=predictions, index=X_validate.index)

new_connections_predictions()

(0, 9)          0.027860
(0, 19)         0.065265
(0, 20)         0.263967
(0, 35)         0.013284
(0, 38)         0.008473
(0, 42)         0.764184
(0, 43)         0.018365
(0, 50)         0.010360
(0, 53)         0.463358
(0, 54)         0.043003
(0, 62)         0.412441
(0, 63)         0.044445
(0, 69)         0.021732
(0, 72)         0.011003
(0, 83)         0.190823
(0, 90)         0.028256
(0, 91)         0.016034
(0, 95)         0.120270
(0, 100)        0.027321
(0, 106)        0.953962
(0, 108)        0.014792
(0, 109)        0.011226
(0, 110)        0.010784
(0, 118)        0.009369
(0, 122)        0.029216
(0, 131)        0.040643
(0, 133)        0.737906
(0, 140)        0.071481
(0, 149)        0.145609
(0, 151)        0.008530
                  ...   
(988, 989)      0.011593
(988, 996)      0.011604
(988, 997)      0.022607
(988, 1000)     0.011586
(988, 1002)     0.011604
(989, 994)      0.011600
(989, 996)      0.011600
(989, 1003)     0.011600
(989, 1004)     0.011600
